In [ ]:
#Snowpark for Python
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.functions import udf
import snowflake.snowpark.functions as F

import numpy as np
#Override np.float_ with np.float64
np.float_ = np.float64

from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.registry import Registry
from snowflake.ml._internal.utils import identifier

# data science libs
import pandas as pd  
# need to add numpy code from previous notebook to handle int64 issue for numpy
import matplotlib.pyplot as plt 
import seaborn as sns 

from snowflake.ml.modeling.metrics import mean_absolute_percentage_error 

# other libs 
import json
import joblib 
import cachetools 

# warning suppression 
import warnings; warnings.simplefilter('ignore')
